Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28, 1) (20000, 10)
Validation set (1000, 28, 28, 1) (1000, 10)
Test set (1000, 28, 28, 1) (1000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
 # if change stride to 1,1,1,1, then must tune layer3_weights to image_size * image_size to maintain valid matrix multiplication
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    #print (shape[0]) # 16, batch_size
    #print (shape[1]) # 7
    #print (shape[2]) # 7
    #print (shape[3]) # 16, depth
    #print(shape[1]*shape[2]*shape[3])
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.20).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.681904
Minibatch accuracy: 0.0%
Validation accuracy: 10.1%
Minibatch loss at step 50: 1.694516
Minibatch accuracy: 50.0%
Validation accuracy: 44.3%
Minibatch loss at step 100: 0.739612
Minibatch accuracy: 62.5%
Validation accuracy: 75.3%
Minibatch loss at step 150: 0.707161
Minibatch accuracy: 75.0%
Validation accuracy: 75.9%
Minibatch loss at step 200: 0.728558
Minibatch accuracy: 75.0%
Validation accuracy: 80.2%
Minibatch loss at step 250: 1.162392
Minibatch accuracy: 62.5%
Validation accuracy: 77.3%
Minibatch loss at step 300: 1.105707
Minibatch accuracy: 62.5%
Validation accuracy: 80.6%
Minibatch loss at step 350: 0.335010
Minibatch accuracy: 87.5%
Validation accuracy: 80.6%
Minibatch loss at step 400: 0.480968
Minibatch accuracy: 93.8%
Validation accuracy: 83.2%
Minibatch loss at step 450: 0.669311
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 500: 0.834699
Minibatch accuracy: 68.8%
Validation accuracy: 82.7%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    pool1 = tf.nn.max_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    bias2 = tf.nn.relu(conv2 + layer2_biases)
    pool2 = tf.nn.max_pool(bias2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.884830
Minibatch accuracy: 25.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.777400
Minibatch accuracy: 31.2%
Validation accuracy: 41.3%
Minibatch loss at step 100: 1.026613
Minibatch accuracy: 62.5%
Validation accuracy: 70.2%
Minibatch loss at step 150: 0.869379
Minibatch accuracy: 75.0%
Validation accuracy: 75.3%
Minibatch loss at step 200: 0.885709
Minibatch accuracy: 62.5%
Validation accuracy: 79.0%
Minibatch loss at step 250: 1.008582
Minibatch accuracy: 62.5%
Validation accuracy: 79.7%
Minibatch loss at step 300: 0.780588
Minibatch accuracy: 68.8%
Validation accuracy: 80.3%
Minibatch loss at step 350: 0.454444
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 400: 0.515155
Minibatch accuracy: 93.8%
Validation accuracy: 80.8%
Minibatch loss at step 450: 0.560428
Minibatch accuracy: 81.2%
Validation accuracy: 82.2%
Minibatch loss at step 500: 0.594460
Minibatch accuracy: 75.0%
Validation accuracy: 81.9%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [9]:
#LeNet

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # C1 input 28 x 28
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    # S2 input 24 x 24
    pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # C3 input 12 x 12
    conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias3 = tf.nn.relu(conv3 + layer2_biases)
    # S4 input 8 x 8
    pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # F6 input 4 x 4
    shape = pool4.get_shape().as_list()
    reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.675638
Minibatch accuracy: 0.0%
Validation accuracy: 8.9%
Minibatch loss at step 50: 1.735252
Minibatch accuracy: 43.8%
Validation accuracy: 44.6%
Minibatch loss at step 100: 1.303931
Minibatch accuracy: 56.2%
Validation accuracy: 67.3%
Minibatch loss at step 150: 1.017518
Minibatch accuracy: 75.0%
Validation accuracy: 73.9%
Minibatch loss at step 200: 1.238529
Minibatch accuracy: 62.5%
Validation accuracy: 76.1%
Minibatch loss at step 250: 1.024834
Minibatch accuracy: 56.2%
Validation accuracy: 74.1%
Minibatch loss at step 300: 1.046636
Minibatch accuracy: 68.8%
Validation accuracy: 75.8%
Minibatch loss at step 350: 0.435839
Minibatch accuracy: 93.8%
Validation accuracy: 77.5%
Minibatch loss at step 400: 0.754883
Minibatch accuracy: 87.5%
Validation accuracy: 77.9%
Minibatch loss at step 450: 0.733302
Minibatch accuracy: 81.2%
Validation accuracy: 78.9%
Minibatch loss at step 500: 0.798144
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%
Min

Minibatch loss at step 4550: 0.201462
Minibatch accuracy: 93.8%
Validation accuracy: 86.2%
Minibatch loss at step 4600: 0.351506
Minibatch accuracy: 87.5%
Validation accuracy: 85.4%
Minibatch loss at step 4650: 0.599993
Minibatch accuracy: 81.2%
Validation accuracy: 84.6%
Minibatch loss at step 4700: 0.085311
Minibatch accuracy: 100.0%
Validation accuracy: 86.3%
Minibatch loss at step 4750: 0.722064
Minibatch accuracy: 75.0%
Validation accuracy: 86.9%
Minibatch loss at step 4800: 0.200005
Minibatch accuracy: 93.8%
Validation accuracy: 85.5%
Minibatch loss at step 4850: 0.630380
Minibatch accuracy: 75.0%
Validation accuracy: 86.4%
Minibatch loss at step 4900: 0.380270
Minibatch accuracy: 93.8%
Validation accuracy: 86.2%
Minibatch loss at step 4950: 0.772563
Minibatch accuracy: 75.0%
Validation accuracy: 86.0%
Minibatch loss at step 5000: 1.965179
Minibatch accuracy: 62.5%
Validation accuracy: 86.8%
Minibatch loss at step 5050: 0.669426
Minibatch accuracy: 81.2%
Validation accuracy: 85.7

Minibatch loss at step 9050: 0.472165
Minibatch accuracy: 93.8%
Validation accuracy: 87.3%
Minibatch loss at step 9100: 0.499862
Minibatch accuracy: 81.2%
Validation accuracy: 87.1%
Minibatch loss at step 9150: 0.613295
Minibatch accuracy: 87.5%
Validation accuracy: 87.6%
Minibatch loss at step 9200: 0.229768
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 9250: 0.377509
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 9300: 0.349189
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 9350: 0.645207
Minibatch accuracy: 68.8%
Validation accuracy: 86.7%
Minibatch loss at step 9400: 0.111464
Minibatch accuracy: 100.0%
Validation accuracy: 87.4%
Minibatch loss at step 9450: 0.502131
Minibatch accuracy: 81.2%
Validation accuracy: 85.6%
Minibatch loss at step 9500: 0.368674
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 9550: 0.359634
Minibatch accuracy: 87.5%
Validation accuracy: 87.1

Minibatch loss at step 13550: 0.045822
Minibatch accuracy: 100.0%
Validation accuracy: 87.5%
Minibatch loss at step 13600: 0.426060
Minibatch accuracy: 93.8%
Validation accuracy: 87.4%
Minibatch loss at step 13650: 0.451267
Minibatch accuracy: 81.2%
Validation accuracy: 86.8%
Minibatch loss at step 13700: 0.260089
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 13750: 0.541972
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 13800: 0.675707
Minibatch accuracy: 81.2%
Validation accuracy: 87.3%
Minibatch loss at step 13850: 0.085899
Minibatch accuracy: 100.0%
Validation accuracy: 86.6%
Minibatch loss at step 13900: 0.156723
Minibatch accuracy: 93.8%
Validation accuracy: 87.3%
Minibatch loss at step 13950: 0.014861
Minibatch accuracy: 100.0%
Validation accuracy: 87.8%
Minibatch loss at step 14000: 0.349735
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Minibatch loss at step 14050: 0.207984
Minibatch accuracy: 87.5%
Validation a

Minibatch loss at step 18000: 0.342841
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 18050: 0.075098
Minibatch accuracy: 100.0%
Validation accuracy: 87.3%
Minibatch loss at step 18100: 0.269481
Minibatch accuracy: 93.8%
Validation accuracy: 86.1%
Minibatch loss at step 18150: 0.241231
Minibatch accuracy: 93.8%
Validation accuracy: 85.9%
Minibatch loss at step 18200: 0.627529
Minibatch accuracy: 93.8%
Validation accuracy: 85.9%
Minibatch loss at step 18250: 0.099211
Minibatch accuracy: 100.0%
Validation accuracy: 86.7%
Minibatch loss at step 18300: 0.060450
Minibatch accuracy: 100.0%
Validation accuracy: 85.4%
Minibatch loss at step 18350: 0.077647
Minibatch accuracy: 100.0%
Validation accuracy: 87.0%
Minibatch loss at step 18400: 0.027565
Minibatch accuracy: 100.0%
Validation accuracy: 86.3%
Minibatch loss at step 18450: 0.093164
Minibatch accuracy: 100.0%
Validation accuracy: 86.5%
Minibatch loss at step 18500: 0.184778
Minibatch accuracy: 93.8%
Validatio